# UrbanSounds8k Data Preprocessing

## Projenin İşleyişi

Öncelikle bize verilen bilgileri almak için not defterini okuyacağım. Bu not defterinde verilerin hangi kriterlere göre
adlandırıldığı belirtilmekte bu kriterlerle beraber sesleri ayırıp farklı klasörlere yerleştireceğim. Böylece projemizin ilk adımı olan veriyi işleme kısmının ilk adımını atmış olacağız. Ardından bu verileri bulundukları klasörde spektogramlara çevireceğim ve takımımızın model oluşturan kısmı için verileri algoritmaya sunulabilir hale getireceğim.

In [9]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import shutil
import csv
import random
with open("UrbanSound8K_README.txt", "r",encoding = "UTF-8") as file:
    readme = file.read()
    print(readme)


UrbanSound8K

Created By
----------

Justin Salamon*^, Christopher Jacoby* and Juan Pablo Bello*
* Music and Audio Research Lab (MARL), New York University, USA
^ Center for Urban Science and Progress (CUSP), New York University, USA
http://serv.cusp.nyu.edu/projects/urbansounddataset
http://marl.smusic.nyu.edu/
http://cusp.nyu.edu/

Version 1.0


Description
-----------

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, 
children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, and street_music. The classes are 
drawn from the urban sound taxonomy described in the following article, which also includes a detailed description of 
the dataset and how it was compiled:

J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 
22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

All excerpts are taken from field recordings uploaded to www.freesou

## ClassID Meanings
[fsID]_[classID]_[occurrenceID]_[sliceID].wav şeklinde adlandırılmış bu dosyalarda class ID'lerini kullanıp veriler üzerinde 
ayırma yapabiliriz.

A numeric identifier of the sound class:

0 = air_conditioner

1 = car_horn

2 = children_playing

3 = dog_bark

4 = drilling

5 = engine_idling

6 = gun_shot

7 = jackhammer

8 = siren

9 = street_music


In [34]:
#İlk olarak fold1 üzerinde çalışacağız bunun için içindeki tüm dosyaların adlarını almak istiyorum. Bunun için ise os.listdir()
#komutunu kullanacağım
#bunun için gerekli kütüphaneyi dahil ediyorum
import os
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/audio/fold1"


filenames = os.listdir(path)
print(filenames)

['.DS_Store', '101415-3-0-2.wav', '101415-3-0-3.wav', '101415-3-0-8.wav', '102106-3-0-0.wav', '102305-6-0-0.wav', '102842-3-0-1.wav', '102842-3-1-0.wav', '102842-3-1-5.wav', '102842-3-1-6.wav', '103074-7-0-0.wav', '103074-7-0-1.wav', '103074-7-0-2.wav', '103074-7-1-0.wav', '103074-7-1-1.wav', '103074-7-1-2.wav', '103074-7-1-3.wav', '103074-7-1-4.wav', '103074-7-1-5.wav', '103074-7-1-6.wav', '103074-7-2-0.wav', '103074-7-3-0.wav', '103074-7-3-1.wav', '103074-7-3-2.wav', '103074-7-3-3.wav', '103074-7-4-0.wav', '103074-7-4-1.wav', '103074-7-4-2.wav', '103074-7-4-3.wav', '103074-7-4-4.wav', '103074-7-4-5.wav', '103074-7-4-6.wav', '103258-5-0-0.wav', '103258-5-0-1.wav', '103258-5-0-10.wav', '103258-5-0-11.wav', '103258-5-0-12.wav', '103258-5-0-13.wav', '103258-5-0-14.wav', '103258-5-0-15.wav', '103258-5-0-16.wav', '103258-5-0-17.wav', '103258-5-0-18.wav', '103258-5-0-19.wav', '103258-5-0-2.wav', '103258-5-0-3.wav', '103258-5-0-4.wav', '103258-5-0-5.wav', '103258-5-0-6.wav', '103258-5-0-7.wa

Artık elimizde fold 1 için tüm dosya isimleri var. Bunları indexleyerek ne sesi olduklarına ulaşabiliriz. Birinci "-" işaretinden hemen sonra gelen sayı bizim bunları sınıflandırmamız için yeterli olacaktır. Oldukları ana klasörün içine ilgili sesleri depolamak için kategorize edilmiş klasörler açacağım. Elimizdeki dosyaların tümünü python yardımı ile bunların içine kategorize edilmiş bir biçimde aktaracağım.

In [35]:
#klasörleri açmak için 
directory_names = ["air_conditioner", "car_horn","children_playing", "dog_bark", "drilling", "engine_idling", "gun_shot", "jackhammer",
                  "siren", "street_music"
                  ]

#ilk olarak processed data klasörünü oluşturalım
directory = "Processed_Data"
parent_dir = "C:/Users/FIRAT/Desktop/UrbanSound8K/"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
print(f"{directory} klasörü oluşturuldu")
for i in directory_names:
    directory = i
    parent_dir = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    print(f"{directory} klasörü oluşturuldu")


Processed_Data klasörü oluşturuldu
air_conditioner klasörü oluşturuldu
car_horn klasörü oluşturuldu
children_playing klasörü oluşturuldu
dog_bark klasörü oluşturuldu
drilling klasörü oluşturuldu
engine_idling klasörü oluşturuldu
gun_shot klasörü oluşturuldu
jackhammer klasörü oluşturuldu
siren klasörü oluşturuldu
street_music klasörü oluşturuldu


In [36]:
#ardından ise elimizdeki fold1 klasörünü inceleyip buradaki koşullara göre veriyi doğru klasörlere aktaralım. Aktarmak için gerekli
#modülü dahil ediyorum ancak öncelikle bir sözlük yapısı oluşturup stabil bir kod yazacağım

import shutil
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
folders = os.listdir(path)
indexes = ["0","1","2","3","4","5","6","7","8","9"]
folder_dict = dict(zip(indexes, folders))
for i in filenames:
    if i.startswith(".DS"):
        continue
    else:
        
        isplitted = i.split("-")
        source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/fold1/{i}"
        destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{folder_dict[isplitted[1]]}"
        shutil.move(source, destination)
        print(f"{i} dosyası başarıyla {destination} klasörüne taşındı")

101415-3-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
101415-3-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
101415-3-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102106-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102305-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
102842-3-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark k

157867-8-0-24.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-25.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-26.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-27.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-28.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157867-8-0-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
15786

196400-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
197073-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
197073-3-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
197073-3-4-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
197073-3-7-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
197318-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
197318-6-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
197318-6-10-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
197318-6-11-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot

Birinci klasördeki verilerin hepsini başarıyla sınıflandırdık. Artık sıra geldi tüm diğer verileri sınıflandırmaya. Bunun için geri kalan klasörlerin hepsinde bir for döngüsü ile gezineceğim ve dosya isimlerini otomatize edip alacağım böylece daha az kod ile daha fazla iş başarabileceğiz.

In [37]:
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/audio/"
unprocessed_file_names = os.listdir(path)
#fold1 klasörünü çıkardım çünkü onu zaten işlemiştik. Şimdi elimizde for döngüsünde kullanabileceğimiz bir liste var
unprocessed_file_names.remove(".DS_Store")
unprocessed_file_names.remove("fold1")
print(unprocessed_file_names)

['fold10', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9']


In [38]:
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
folders = os.listdir(path)
indexes = ["0","1","2","3","4","5","6","7","8","9"]
folder_dict = dict(zip(indexes, folders))
for i in unprocessed_file_names:
    datapath = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/{i}"
    filenames2 = os.listdir(datapath)
    
    for j in filenames2:
        if j.startswith(".DS"):
            continue
        else:
            
            jsplitted = j.split("-")
           
            source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/{i}/{j}"
            destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{folder_dict[jsplitted[1]]}"
            shutil.move(source, destination)
            print(f"{j} dosyası başarıyla {destination} klasörüne taşındı")
            

100648-1-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-4-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100795-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark k

165640-4-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
165640-4-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
167464-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-14.wav dosyası başarıyla C:/Users/FIRAT

71312-3-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72220-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72220-3-2-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72220-3-2-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72220-3-2-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72538-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72538-3-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72538-3-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72538-3-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne 

143970-5-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
143970-5-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
143970-5-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
145577-1-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
145608-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
145608-6-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
145608-6-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
145608-6-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
146690-0-0-100.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processe

189023-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
189023-0-0-6.wav dosyası başarıyla C

...

125678-7-3-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-3-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-3-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-3-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-3-7.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-4-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-4-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-4-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
125678-7-4-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed

180134-4-1-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-15.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-16.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-17.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling klasörüne taşındı
180134-4-2-19.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dril

80589-0-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
80589-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
80589-0-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
81787-2-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
81787-2-0-14.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
81787-2-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
81787-2-0-35.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
81787-2-0-39.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
81787-2-0-4.wav dosyası başarıyla C:/Us

157866-8-0-27.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-7.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
157866-8-0-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
159707-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
159726

39856-5-0-21.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-22.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-24.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-25.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-26.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-27.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-28.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
39856-5-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop

Böylece verimizi tiplerine göre ayırmış olduk. Bu ayrım onları etiketlememiz açısından bize fayda sağlayacak. Yapmamız gereken bir ayırma işlemi daha var. README dosyasında görüldüğü üzere sesler arkaplan ve önden gelen sesler olarak ikiye ayrılmış. Model oluşturma açısından ekibime kolaylık olması için ben bu ayrımı da yapacağım.

1 = Foreground

2= background 

şeklinde nitelenmiş bu verilere ulaşıp onları kendi klasörlerinde ikiye ayıracağım.


In [39]:
list_dir_processed = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data")
print(list_dir_processed)

['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']


In [40]:
#Bu veriyi almak için veri setinin bize sunduğu csv dosyasını okumamız gerekmekte bunun için ise pandas kullanacağım
dataset = pd.read_csv("C:/Users/FIRAT/Desktop/UrbanSound8K/metadata/UrbanSound8K.csv")
dataset.head()
foregroundnoises = dataset[dataset["salience"] == 1]
foregroundnoises_names =list(foregroundnoises["slice_file_name"])
backgroundnoises = dataset[dataset["salience"] == 2]
backgroundnoises_names = list(backgroundnoises["slice_file_name"])


In [41]:
for i in list_dir_processed:
    path = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}"
    filenames = os.listdir(path)
    
    
    directory = "Foreground"
    
    path = os.path.join(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/", directory)
    os.mkdir(path)
    print(f"{directory} klasörü {i} konumunda oluşturuldu oluşturuldu")
    directory2 = "Background"
    
    path = os.path.join(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/", directory2)
    os.mkdir(path)
    print(f"{directory} klasörü {i} konumunda oluşturuldu oluşturuldu")
    
    for j in filenames:
        if j in foregroundnoises_names:
            source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/{j}"
            destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Foreground"
            shutil.move(source, destination)
            print(f"{j} dosyası başarıyla {destination} konumuna taşındı")
        elif j in backgroundnoises_names:
            source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/{j}"
            destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Background"
            shutil.move(source, destination)
            print(f"{j} dosyası başarıyla {destination} konumuna taşındı")
    

Foreground klasörü air_conditioner konumunda oluşturuldu oluşturuldu
Foreground klasörü air_conditioner konumunda oluşturuldu oluşturuldu
100852-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
100852-0-0-14.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Proces

153261-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
153261-0-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
153261-0-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
153261-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
159761-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumuna taşındı
159761-0-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumuna taşındı
159761-0-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumuna taşındı
159761-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_

189981-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-14.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-15.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-16.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
189981-0-0-17.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_D

74507-0-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
74507-0-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
74507-0-0-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
74507-0-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
74507-0-0-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumuna taşındı
74677-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumuna taşındı
74677-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumuna taşındı
74677-0-0-105.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_condi

24074-1-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground konumuna taşındı
24074-1-0-6.wav dosy

...

57323-8-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
57323-8-0-6.wav dosyası başarıyla C:/Users/FIR

93567-8-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
93567-8-0-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94020-8-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94020-8-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Foreground konumuna taşındı
94636-8-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94636-8-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94636-8-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94636-8-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background konumuna taşındı
94636-8-0-12.wav dosyası başarıyla C:/Users/FI

139951-9-0-17.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
139951-9-0-29.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
139951-9-0-33.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
139951-9-0-45.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
139951-9-0-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
14385-9-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
14385-9-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
14385-9-0-13.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foregrou

185374-9-0-60.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
185375-9-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
185375-9-0-26.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
185375-9-0-33.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
185375-9-0-60.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
185375-9-0-61.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
185375-9-0-84.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
186336-9-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Backg

73277-9-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
73277-9-0-16.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
73277-9-0-17.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
73277-9-0-19.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
73277-9-0-24.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
73277-9-0-28.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground konumuna taşındı
7390-9-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumuna taşındı
7390-9-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background konumu

Böylece elimizdeki veri setinin tamamını türlerine ve arkaplandan ya da önden gelmesine göre ayırmış olduk. Bu analizimizi daha da kolaylaştıracaktır. Şimdi sıra bu verilerin tamamını spektogram haline çevirip görüntü işlemeye hazır hale getirmekte.

In [42]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [49]:
#Şimdi bu wav dosyalarını spektrogram şeklinde kaydetmek için gerekli kodlarımızı yazalım

def create_spectrogram(y):
    spec = librosa.feature.melspectrogram(y=y)
    spec_conv = librosa.amplitude_to_db(spec, ref =np.max)
    return spec_conv
for i in list_dir_processed:
    path_foreground = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Foreground"
    filenames1 = os.listdir(path_foreground)
    path_background = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Background"
    filenames2 = os.listdir(path_background)
    directory = "Spectrograms"
    folderpath = os.path.join(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Foreground/", directory)
    os.mkdir(folderpath)
    print(f"{directory} klasörü {path_foreground} konumunda oluşturuldu oluşturuldu")
    directory = "Spectrograms"
    folderpath2 = os.path.join(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/Background/", directory)
    os.mkdir(folderpath2)
    print(f"{directory} klasörü {path_background} konumunda oluşturuldu oluşturuldu")
    paths_list = [path_foreground, path_background]
    
    
    for j in filenames1:
        path_of_file = path_foreground + f"/{j}"
        y, sr = librosa.load(path_of_file)
        spectrogram = create_spectrogram(y)
        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        p = librosa.display.specshow(spectrogram, ax=ax, y_axis='log', x_axis='time')
        new_filename = j.replace(".wav",".png")
        fig.savefig(f"{path_foreground}/Spectrograms/{new_filename}")
        
        
        print(f"Spectrogram başarıyla kaydedildi {new_filename}")
        
    for j in filenames2:
        path_of_file = path_background + f"/{j}"
        y, sr = librosa.load(path_of_file)
        spectrogram = create_spectrogram(y)
        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        p = librosa.display.specshow(spectrogram, ax=ax, y_axis='log', x_axis='time')
        new_filename = j.replace(".wav",".png")
        fig.savefig(f"{path_background}/Spectrograms/{new_filename}")
        
        print(f"Spectrogram başarıyla kaydedildi {new_filename}")
        
    


Spectrograms klasörü C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground konumunda oluşturuldu oluşturuldu
Spectrograms klasörü C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background konumunda oluşturuldu oluşturuldu
Spectrogram başarıyla kaydedildi 100852-0-0-0.png
Spectrogram başarıyla kaydedildi 100852-0-0-1.png
Spectrogram başarıyla kaydedildi 100852-0-0-10.png
Spectrogram başarıyla kaydedildi 100852-0-0-11.png
Spectrogram başarıyla kaydedildi 100852-0-0-12.png
Spectrogram başarıyla kaydedildi 100852-0-0-13.png
Spectrogram başarıyla kaydedildi 100852-0-0-14.png
Spectrogram başarıyla kaydedildi 100852-0-0-15.png
Spectrogram başarıyla kaydedildi 100852-0-0-16.png
Spectrogram başarıyla kaydedildi 100852-0-0-17.png
Spectrogram başarıyla kaydedildi 100852-0-0-18.png
Spectrogram başarıyla kaydedildi 100852-0-0-19.png
Spectrogram başarıyla kaydedildi 100852-0-0-2.png
Spectrogram başarıyla kaydedildi 100852-0-0-20.png
Spectrogram başarıyla k

Spectrogram başarıyla kaydedildi 146709-0-0-66.png
Spectrogram başarıyla kaydedildi 146709-0-0-68.png
Spectrogram başarıyla kaydedildi 146709-0-0-7.png
Spectrogram başarıyla kaydedildi 146845-0-0-0.png
Spectrogram başarıyla kaydedildi 146845-0-0-1.png
Spectrogram başarıyla kaydedildi 146845-0-0-10.png
Spectrogram başarıyla kaydedildi 146845-0-0-11.png
Spectrogram başarıyla kaydedildi 146845-0-0-12.png
Spectrogram başarıyla kaydedildi 146845-0-0-13.png
Spectrogram başarıyla kaydedildi 146845-0-0-14.png
Spectrogram başarıyla kaydedildi 146845-0-0-15.png
Spectrogram başarıyla kaydedildi 146845-0-0-16.png
Spectrogram başarıyla kaydedildi 146845-0-0-17.png
Spectrogram başarıyla kaydedildi 146845-0-0-18.png
Spectrogram başarıyla kaydedildi 146845-0-0-19.png
Spectrogram başarıyla kaydedildi 146845-0-0-2.png
Spectrogram başarıyla kaydedildi 146845-0-0-20.png
Spectrogram başarıyla kaydedildi 146845-0-0-21.png
Spectrogram başarıyla kaydedildi 146845-0-0-22.png
Spectrogram başarıyla kaydedildi 14

Spectrogram başarıyla kaydedildi 189981-0-0-5.png
Spectrogram başarıyla kaydedildi 189981-0-0-6.png
Spectrogram başarıyla kaydedildi 189981-0-0-7.png
Spectrogram başarıyla kaydedildi 189981-0-0-8.png
Spectrogram başarıyla kaydedildi 189981-0-0-9.png
Spectrogram başarıyla kaydedildi 189982-0-0-11.png
Spectrogram başarıyla kaydedildi 189982-0-0-13.png
Spectrogram başarıyla kaydedildi 189982-0-0-16.png
Spectrogram başarıyla kaydedildi 189982-0-0-17.png
Spectrogram başarıyla kaydedildi 189982-0-0-18.png
Spectrogram başarıyla kaydedildi 189982-0-0-19.png
Spectrogram başarıyla kaydedildi 189982-0-0-2.png
Spectrogram başarıyla kaydedildi 189982-0-0-20.png
Spectrogram başarıyla kaydedildi 189982-0-0-23.png
Spectrogram başarıyla kaydedildi 189982-0-0-27.png
Spectrogram başarıyla kaydedildi 189982-0-0-28.png
Spectrogram başarıyla kaydedildi 189982-0-0-3.png
Spectrogram başarıyla kaydedildi 189982-0-0-30.png
Spectrogram başarıyla kaydedildi 189982-0-0-33.png
Spectrogram başarıyla kaydedildi 18998

Spectrogram başarıyla kaydedildi 55018-0-0-92.png
Spectrogram başarıyla kaydedildi 55018-0-0-93.png
Spectrogram başarıyla kaydedildi 58806-0-0-0.png
Spectrogram başarıyla kaydedildi 59277-0-0-0.png
Spectrogram başarıyla kaydedildi 59277-0-0-1.png
Spectrogram başarıyla kaydedildi 59277-0-0-2.png
Spectrogram başarıyla kaydedildi 59277-0-0-3.png
Spectrogram başarıyla kaydedildi 59277-0-0-4.png
Spectrogram başarıyla kaydedildi 59277-0-0-5.png
Spectrogram başarıyla kaydedildi 59277-0-0-6.png
Spectrogram başarıyla kaydedildi 59277-0-0-7.png
Spectrogram başarıyla kaydedildi 60846-0-0-0.png
Spectrogram başarıyla kaydedildi 60846-0-0-1.png
Spectrogram başarıyla kaydedildi 60846-0-0-2.png
Spectrogram başarıyla kaydedildi 60846-0-0-3.png
Spectrogram başarıyla kaydedildi 62461-0-0-0.png
Spectrogram başarıyla kaydedildi 62461-0-0-1.png
Spectrogram başarıyla kaydedildi 62461-0-0-2.png
Spectrogram başarıyla kaydedildi 62461-0-0-3.png
Spectrogram başarıyla kaydedildi 73524-0-0-0.png
Spectrogram başarı

Spectrogram başarıyla kaydedildi 167464-0-0-4.png
Spectrogram başarıyla kaydedildi 167464-0-0-5.png
Spectrogram başarıyla kaydedildi 167464-0-0-6.png
Spectrogram başarıyla kaydedildi 167464-0-0-7.png
Spectrogram başarıyla kaydedildi 167464-0-0-8.png
Spectrogram başarıyla kaydedildi 167464-0-0-9.png
Spectrogram başarıyla kaydedildi 177621-0-0-0.png
Spectrogram başarıyla kaydedildi 177621-0-0-100.png
Spectrogram başarıyla kaydedildi 177621-0-0-104.png
Spectrogram başarıyla kaydedildi 177621-0-0-105.png
Spectrogram başarıyla kaydedildi 177621-0-0-109.png
Spectrogram başarıyla kaydedildi 177621-0-0-117.png
Spectrogram başarıyla kaydedildi 177621-0-0-120.png
Spectrogram başarıyla kaydedildi 177621-0-0-126.png
Spectrogram başarıyla kaydedildi 177621-0-0-127.png
Spectrogram başarıyla kaydedildi 177621-0-0-128.png
Spectrogram başarıyla kaydedildi 177621-0-0-145.png
Spectrogram başarıyla kaydedildi 177621-0-0-147.png
Spectrogram başarıyla kaydedildi 177621-0-0-2.png
Spectrogram başarıyla kayded

...

Spectrogram başarıyla kaydedildi 173891-9-0-2.png
Spectrogram başarıyla kaydedildi 173891-9-0-7.png
Spectrogram başarıyla kaydedildi 178099-9-0-0.png
Spectrogram başarıyla kaydedildi 178099-9-0-1.png
Spectrogram başarıyla kaydedildi 178099-9-0-10.png
Spectrogram başarıyla kaydedildi 178099-9-0-12.png
Spectrogram başarıyla kaydedildi 178099-9-0-6.png
Spectrogram başarıyla kaydedildi 178099-9-0-7.png
Spectrogram başarıyla kaydedildi 181102-9-0-111.png
Spectrogram başarıyla kaydedildi 181102-9-0-117.png
Spectrogram başarıyla kaydedildi 181102-9-0-23.png
Spectrogram başarıyla kaydedildi 181102-9-0-26.png
Spectrogram başarıyla kaydedildi 181102-9-0-50.png
Spectrogram başarıyla kaydedildi 181102-9-0-97.png
Spectrogram başarıyla kaydedildi 182103-9-0-0.png
Spectrogram başarıyla kaydedildi 182103-9-0-22.png
Spectrogram başarıyla kaydedildi 182103-9-0-26.png
Spectrogram başarıyla kaydedildi 182103-9-0-47.png
Spectrogram başarıyla kaydedildi 182103-9-0-53.png
Spectrogram başarıyla kaydedildi 182

Spectrogram başarıyla kaydedildi 42955-9-0-14.png
Spectrogram başarıyla kaydedildi 42955-9-0-18.png
Spectrogram başarıyla kaydedildi 42955-9-0-19.png
Spectrogram başarıyla kaydedildi 42955-9-0-22.png
Spectrogram başarıyla kaydedildi 49485-9-0-138.png
Spectrogram başarıyla kaydedildi 49485-9-0-142.png
Spectrogram başarıyla kaydedildi 49485-9-0-154.png
Spectrogram başarıyla kaydedildi 49485-9-0-24.png
Spectrogram başarıyla kaydedildi 49485-9-0-28.png
Spectrogram başarıyla kaydedildi 49485-9-0-91.png
Spectrogram başarıyla kaydedildi 55728-9-0-14.png
Spectrogram başarıyla kaydedildi 55728-9-0-28.png
Spectrogram başarıyla kaydedildi 55728-9-0-30.png
Spectrogram başarıyla kaydedildi 55728-9-0-34.png
Spectrogram başarıyla kaydedildi 55728-9-0-67.png
Spectrogram başarıyla kaydedildi 55728-9-0-8.png
Spectrogram başarıyla kaydedildi 60605-9-0-34.png
Spectrogram başarıyla kaydedildi 60605-9-0-52.png
Spectrogram başarıyla kaydedildi 60605-9-0-64.png
Spectrogram başarıyla kaydedildi 60605-9-0-73.pn

Spectrogram başarıyla kaydedildi 99710-9-0-11.png
Spectrogram başarıyla kaydedildi 99710-9-0-12.png
Spectrogram başarıyla kaydedildi 99710-9-0-16.png
Spectrogram başarıyla kaydedildi 99710-9-0-2.png
Spectrogram başarıyla kaydedildi 99710-9-0-5.png
Spectrogram başarıyla kaydedildi 99710-9-0-6.png
Spectrogram başarıyla kaydedildi 107653-9-0-12.png
Spectrogram başarıyla kaydedildi 107653-9-0-14.png
Spectrogram başarıyla kaydedildi 107653-9-0-18.png
Spectrogram başarıyla kaydedildi 107653-9-0-3.png
Spectrogram başarıyla kaydedildi 107653-9-0-5.png
Spectrogram başarıyla kaydedildi 107653-9-0-7.png
Spectrogram başarıyla kaydedildi 113601-9-0-22.png
Spectrogram başarıyla kaydedildi 113601-9-0-3.png
Spectrogram başarıyla kaydedildi 113601-9-0-33.png
Spectrogram başarıyla kaydedildi 113601-9-0-34.png
Spectrogram başarıyla kaydedildi 113601-9-0-35.png
Spectrogram başarıyla kaydedildi 113601-9-0-42.png
Spectrogram başarıyla kaydedildi 115415-9-0-0.png
Spectrogram başarıyla kaydedildi 115415-9-0-1

Spectrogram başarıyla kaydedildi 180057-9-0-7.png
Spectrogram başarıyla kaydedildi 185373-9-0-21.png
Spectrogram başarıyla kaydedildi 185373-9-0-6.png
Spectrogram başarıyla kaydedildi 185373-9-1-17.png
Spectrogram başarıyla kaydedildi 185373-9-1-46.png
Spectrogram başarıyla kaydedildi 185373-9-1-7.png
Spectrogram başarıyla kaydedildi 185373-9-1-70.png
Spectrogram başarıyla kaydedildi 185375-9-0-23.png
Spectrogram başarıyla kaydedildi 185375-9-0-26.png
Spectrogram başarıyla kaydedildi 185375-9-0-33.png
Spectrogram başarıyla kaydedildi 185375-9-0-60.png
Spectrogram başarıyla kaydedildi 185375-9-0-61.png
Spectrogram başarıyla kaydedildi 185375-9-0-84.png
Spectrogram başarıyla kaydedildi 186336-9-0-0.png
Spectrogram başarıyla kaydedildi 186336-9-0-1.png
Spectrogram başarıyla kaydedildi 186336-9-0-2.png
Spectrogram başarıyla kaydedildi 186336-9-0-3.png
Spectrogram başarıyla kaydedildi 186336-9-0-4.png
Spectrogram başarıyla kaydedildi 186339-9-0-1.png
Spectrogram başarıyla kaydedildi 186339-

Spectrogram başarıyla kaydedildi 79377-9-0-13.png
Spectrogram başarıyla kaydedildi 79377-9-0-2.png
Spectrogram başarıyla kaydedildi 79377-9-0-4.png
Spectrogram başarıyla kaydedildi 79377-9-0-8.png
Spectrogram başarıyla kaydedildi 83191-9-0-0.png
Spectrogram başarıyla kaydedildi 83191-9-0-11.png
Spectrogram başarıyla kaydedildi 83191-9-0-13.png
Spectrogram başarıyla kaydedildi 83191-9-0-17.png
Spectrogram başarıyla kaydedildi 83191-9-0-4.png
Spectrogram başarıyla kaydedildi 83191-9-0-6.png
Spectrogram başarıyla kaydedildi 83195-9-0-11.png
Spectrogram başarıyla kaydedildi 83195-9-0-14.png
Spectrogram başarıyla kaydedildi 83195-9-0-2.png
Spectrogram başarıyla kaydedildi 83195-9-0-3.png
Spectrogram başarıyla kaydedildi 83195-9-0-6.png
Spectrogram başarıyla kaydedildi 83195-9-0-7.png
Spectrogram başarıyla kaydedildi 83199-9-0-0.png
Spectrogram başarıyla kaydedildi 84249-9-0-10.png
Spectrogram başarıyla kaydedildi 84249-9-0-11.png
Spectrogram başarıyla kaydedildi 84249-9-0-3.png
Spectrogram 

# Creating a Tagged Data List
Bundan sonrasında ise veriyi bölmek ve modele sunmaya hazır hale getirmek için tüm veriyi etiketleyip kaydedeceğim, böylece veri proje arkadaşlarımın bilgisayarlarında da direkt olarak okunabilir hale gelecektir.


In [2]:
air_conditioner_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Background/Spectrograms")
air_conditioner_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Foreground/Spectrograms"))
car_horn_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Background/Spectrograms")
car_horn_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Foreground/Spectrograms"))
childeren_playing_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/childeren_playing/Background/Spectrograms")
childeren_playing_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/childeren_playing/Foreground/Spectrograms"))
dog_bark_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark/Background/Spectrograms")
dog_bark_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark/Foreground/Spectrograms"))
drilling_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling/Background/Spectrograms")
drilling_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling/Foreground/Spectrograms"))
engine_idling_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling/Background/Spectrograms")
engine_idling_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling/Foreground/Spectrograms"))
gun_shot_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot/Background/Spectrograms")
gun_shot_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot/Foreground/Spectrograms"))
jackhammer_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer/Background/Spectrograms")
jackhammer_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer/Foreground/Spectrograms"))
siren_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Background/Spectrograms")
siren_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Foreground/Spectrograms"))
street_music_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Background/Spectrograms")
street_music_sounds.extend(os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Foreground/Spectrograms"))
tagged_data = []

for i in air_conditioner_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("air_conditioner")
    tagged_data.append(pretagged)
for i in car_horn_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("car_horn")
    tagged_data.append(pretagged)
for i in childeren_playing_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("childeren_playing")
    tagged_data.append(pretagged)
for i in dog_bark_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("dog_bark")
    tagged_data.append(pretagged)
for i in drilling_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("drilling")
    tagged_data.append(pretagged)
for i in engine_idling_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("engine_idling")
    tagged_data.append(pretagged)
for i in gun_shot_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("gun_shot")
    tagged_data.append(pretagged)
for i in jackhammer_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("jackhammer")
    tagged_data.append(pretagged)
for i in siren_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("siren")
    tagged_data.append(pretagged)
for i in street_music_sounds:
    pretagged = []
    pretagged.append(i)
    pretagged.append("street_music")
    tagged_data.append(pretagged)

#tüm dosya isimlerini etiketleriyle birlikte birleştirerek bir listeye aktardım
#şimdi bu listeye bakalım
print(tagged_data)

[['101729-0-0-1.png', 'air_conditioner'], ['101729-0-0-11.png', 'air_conditioner'], ['101729-0-0-12.png', 'air_conditioner'], ['101729-0-0-13.png', 'air_conditioner'], ['101729-0-0-14.png', 'air_conditioner'], ['101729-0-0-16.png', 'air_conditioner'], ['101729-0-0-17.png', 'air_conditioner'], ['101729-0-0-18.png', 'air_conditioner'], ['101729-0-0-19.png', 'air_conditioner'], ['101729-0-0-21.png', 'air_conditioner'], ['101729-0-0-22.png', 'air_conditioner'], ['101729-0-0-23.png', 'air_conditioner'], ['101729-0-0-24.png', 'air_conditioner'], ['101729-0-0-26.png', 'air_conditioner'], ['101729-0-0-28.png', 'air_conditioner'], ['101729-0-0-29.png', 'air_conditioner'], ['101729-0-0-3.png', 'air_conditioner'], ['101729-0-0-32.png', 'air_conditioner'], ['101729-0-0-33.png', 'air_conditioner'], ['101729-0-0-36.png', 'air_conditioner'], ['101729-0-0-37.png', 'air_conditioner'], ['101729-0-0-38.png', 'air_conditioner'], ['101729-0-0-39.png', 'air_conditioner'], ['101729-0-0-4.png', 'air_condition

Bu isimleri başka bilgisayarda da sürekli kullanılabilir hale getirmek için bir csv dosyasına yazmak istiyorum böylece kullanacak diğer bilgisayarlar bunu hemen alabilir ve üzerinde çalışmaya başlayabilir.


In [6]:


with open("tagged_data.csv","w+") as csv_file:
    csvWriter = csv.writer(csv_file,delimiter=',')
    csvWriter.writerows(tagged_data)

Böylece geriye veriyi sadece ayırıp farklı farklı csvler şeklinde diğer bilgisayarlarda açılabilir bir formatta sunmak kaldı. Bunun için X ve y şeklinde ayırıp bunların alt kümeleri olarak train, validation ve test kümeleri oluşturacağım. Böylece veri modele sunulmaya hazır hale gelecektir.

In [8]:
#csv dosyasından veriyi liste olarak almak için kaynak kod
with open("tagged_data.csv", "r", encoding = "UTF-8") as file:
    tagged_data_list = file.readlines()
    
    tagged_data_list = set(tagged_data_list)
    tagged_data = []
    tagged_data_list.remove("\n")
    for i in tagged_data_list:
        prelist = i.split(",")
        prelist[1] = prelist[1].strip("\n")
        tagged_data.append(prelist)
    print(tagged_data)

[['162431-6-2-0.png', 'gun_shot'], ['169098-7-0-6.png', 'jackhammer'], ['104817-4-0-10.png', 'drilling'], ['118279-8-0-12.png', 'siren'], ['42097-7-0-0.png', 'jackhammer'], ['156634-5-0-5.png', 'engine_idling'], ['74677-0-0-10.png', 'air_conditioner'], ['162432-6-1-0.png', 'gun_shot'], ['74726-8-2-2.png', 'siren'], ['17853-5-0-5.png', 'engine_idling'], ['162431-6-0-0.png', 'gun_shot'], ['172314-9-0-51.png', 'street_music'], ['204240-0-0-17.png', 'air_conditioner'], ['76085-4-0-62.png', 'drilling'], ['162134-7-8-0.png', 'jackhammer'], ['197318-6-4-0.png', 'gun_shot'], ['54383-0-0-5.png', 'air_conditioner'], ['30206-7-0-12.png', 'jackhammer'], ['72579-3-0-4.png', 'dog_bark'], ['162432-6-3-0.png', 'gun_shot'], ['74513-3-0-6.png', 'dog_bark'], ['192123-2-0-18.png', 'childeren_playing'], ['185909-2-0-22.png', 'childeren_playing'], ['155280-2-0-15.png', 'childeren_playing'], ['157867-8-0-23.png', 'siren'], ['62461-0-0-2.png', 'air_conditioner'], ['105289-8-0-4.png', 'siren'], ['111386-5-1-4.

In [11]:
#Veriyi yanlı olmadan bölmek açısından karıştırmak için random kütüphanesi içerisindeki shuffle metodunu kullanacağım.
random.shuffle(tagged_data)
print(tagged_data)

[['193698-2-0-64.png', 'childeren_playing'], ['30204-0-0-8.png', 'air_conditioner'], ['159761-0-0-0.png', 'air_conditioner'], ['37869-3-0-0.png', 'dog_bark'], ['155217-9-0-85.png', 'street_music'], ['188823-7-4-0.png', 'jackhammer'], ['72259-1-7-12.png', 'car_horn'], ['99812-1-0-1.png', 'car_horn'], ['193699-2-0-109.png', 'childeren_playing'], ['180977-3-1-5.png', 'dog_bark'], ['146690-0-0-155.png', 'air_conditioner'], ['171305-7-19-0.png', 'jackhammer'], ['125678-7-2-6.png', 'jackhammer'], ['184805-0-0-82.png', 'air_conditioner'], ['77751-4-9-0.png', 'drilling'], ['156893-7-5-0.png', 'jackhammer'], ['164194-2-0-29.png', 'childeren_playing'], ['72015-2-0-2.png', 'childeren_playing'], ['94632-5-0-21.png', 'engine_idling'], ['36429-2-0-13.png', 'childeren_playing'], ['194841-9-0-178.png', 'street_music'], ['180937-4-0-20.png', 'drilling'], ['178260-7-3-8.png', 'jackhammer'], ['147926-0-0-27.png', 'air_conditioner'], ['74726-8-2-1.png', 'siren'], ['71700-3-4-0.png', 'dog_bark'], ['66622-4